In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from statsmodels.formula.api import ols

In [2]:
RDL = pd.read_excel("/Users/moca_lucy/Documents/大學/專題/RDL_AlCu/RDL_biweekly+four-weekly+monthly.xlsx", sheet_name="Biweekly", header=0)

In [19]:
traindata = RDL[0:24]
testdata = RDL[24:]
testdata

,Biweek,X1,X2,X3,X4,X5,X6,X13,X14,X15,Y
24,2021-18,1023,0.1405,3075,0.1049,7422,0.0931,96.3039,83.6869,1,8
25,2021-20,1298,0.1751,10068,0.1617,9002,0.2317,98.4296,89.7086,1,19
26,2021-22,424,0.0722,2329,0.0754,11449,0.0895,97.8887,90.1226,1,10
27,2021-24,519,0.1472,5737,0.0728,4688,0.0750,95.4330,85.7408,1,10
28,2021-26,854,0.0756,3564,0.0537,4822,0.0493,95.2179,87.9899,1,7
29,2021-28,2552,0.0998,3814,0.0759,6844,0.1077,94.1363,90.5835,1,10


In [20]:
regression = ols(formula='Y~X1+X2+X3+X4+X5+X6+X13+X14+X15', data=traindata)
lm = regression.fit()

In [21]:
#lm.summary()

In [22]:
#lm.params

In [23]:
'''print('Y = ', end='')
for index, value in enumerate(lm.params):
    if index==0:
        print(value, end='')
    else:
        if value>0:
            print('+', value, 'X', index, end='')
        else:
            print(value, 'X', index, end='')'''

"print('Y = ', end='')\nfor index, value in enumerate(lm.params):\n    if index==0:\n        print(value, end='')\n    else:\n        if value>0:\n            print('+', value, 'X', index, end='')\n        else:\n            print(value, 'X', index, end='')"

In [24]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true-y_pred)/y_true))*100

In [25]:
def mase(y_true, y_pred):
    y_num = y_true.size
    d = np.abs(np.diff(y_true)).sum()/(y_num-1)
    ret = np.abs(y_true - y_pred)
    return ret.mean()/d

In [26]:
def table(lm, data, w):
    y_pred = round(lm.predict(data), 0)
    y_true = data["Y"]
   # print(y_true[190])
    #print(np.where(np.isnan(y_pred)))
    #print(y_pred, y_true)
    MAE = round(metrics.mean_absolute_error(y_true, y_pred), 4)
    #print(metrics.mean_absolute_error(y_true, y_pred))
    MSE = round(metrics.mean_squared_error(y_true, y_pred), 4)
    RMSE = round(np.sqrt(metrics.mean_squared_error(y_true, y_pred)), 4)
    try:
        MAPE =  str(round(mape(y_true, y_pred), 4))+'%'
    except ZeroDivisionError:
        MAPE = "NAN"
    MASE = round(mase(y_true, y_pred), 4)
    errortype = ["MAE", "MSE", "RMSE", "MAPE", "MASE"]
    error = [MAE, MSE, RMSE, MAPE, MASE]
    #print(error)
    if w == "train":
        error_dict = {" ":errortype, "training":error}
    elif w == "test":
        error_dict = {" ":errortype, "testing":error}
    #error_dict = {" ":errortype, "誤差分析":error}
    pd.options.display.float_format = '{:.4f}'.format
    df = pd.DataFrame(error_dict)
    df = df.set_index(" ").rename_axis(None)
    #print(df)
    return df

df1 = table(lm, traindata, "train")
df1

,training
MAE,0.9583
MSE,1.6250
RMSE,1.2748
MAPE,9.0966%
MASE,0.2422


In [27]:
df2 = table(lm, testdata, "test")
df2

,testing
MAE,1.0000
MSE,1.6667
RMSE,1.2910
MAPE,11.5476%
MASE,0.1923
